In [42]:
import sys

sys.path.append("../")

import pandas as pd
from dotenv import load_dotenv

load_dotenv()
import os
import sqlite3

DB_PATH = os.getenv("DB_PATH")
DATA_PATH = "data"

conn = sqlite3.connect(DB_PATH)

if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

In [43]:
# Individuals Regions

df = pd.read_csv("../immaterial_index/results/df_individuals_score.csv")
print(len(set(df.individual_wikidata_id)))

df[df['individual_name']=='Hesiod']

159340


,Unnamed: 0,individual_wikidata_id,productive_year,decade,individual_name,score,region_name
297556,312663,Q44233,-715.0,-720,Hesiod,147,Eastern Europe
297557,312664,Q44233,-715.0,-720,Hesiod,147,Balkans
297558,312665,Q44233,-715.0,-720,Hesiod,147,mediterranean World
297559,312666,Q44233,-715.0,-720,Hesiod,147,Greek World


In [44]:


# Load works of individuals

df_ind_works = pd.read_sql_query("SELECT * FROM individual_created_work", conn)

df_count_work = (
    df_ind_works.groupby("individual_wikidata_id")["work_wikidata_id"]
    .count()
    .rename("count_works")
    .reset_index()
)
df_final = pd.merge(df, df_count_work, on="individual_wikidata_id", how="left")
df_final = df_final.fillna(0)  # When there is no works we add 0
df_final = df_final.drop('Unnamed: 0', axis=1)
df_final = df_final.drop('productive_year', axis=1)
df_final.to_csv(DATA_PATH + "/df_indi_works.csv")

In [45]:
df_gender = pd.read_sql_query("SELECT * FROM individual_gender", conn)
df_final_gender = pd.merge(df_gender[['individual_wikidata_id', 'gender']], df_final, on = 'individual_wikidata_id')
df_final_gender.to_csv(DATA_PATH + "/df_indi_works_gender.csv")

In [46]:

   # "source": [
   #    "# Individuals Regions\n",
   #    "\n",
   #    "df = pd.read_csv(\"../immaterial_index/results/df_individuals_score.csv\")\n",
   #    "print(len(set(df.individual_wikidata_id)))\n",
   #    "\n",
   #    "df[df['individual_name']=='Hesiod']"
   # ]

## Associate a category to CPs based on the type of their work

In [47]:
df_work_type = pd.read_sql_query("SELECT * FROM created_work", conn)

In [48]:
df_work_category = df_work_type[['work_wikidata_id', 'instance_label']].drop_duplicates()
df_work_covariate = pd.merge(df_ind_works, df_work_category, on = ['work_wikidata_id'])
df_work_covariate['instance_label'].value_counts().head(20)

# took the top 20 categories that occur the most and group them

# Define mapping of original categories to new categories
category_mapping = {
    'painting': 'painting',
    'biographical article': 'literary work',
    'poem': 'literary work',
    'print': 'print',
    'literary work': 'literary work',
    'version, edition, or translation': 'literary work',
    'drawing': 'drawing',
    'encyclopedia article': 'literary work',
    'article': 'literary work',
    'photograph': 'photograph',
    'publication': 'literary work',
    'watercolor painting': 'painting',
    'scholarly article': 'literary work',
    'sculpture': 'sculpture',
    'engraving': 'engraving',
    'poetry': 'literary work',
    'written work': 'literary work',
    'etching print': 'print',
}
df_work_covariate['category']  = df_work_covariate['instance_label'].apply(lambda x:category_mapping.get(x, None)) # drop None otherwise
df_work_covariate = df_work_covariate.dropna()
df_work_covariate = df_work_covariate[['individual_wikidata_id', 'work_wikidata_id', 'category']].drop_duplicates()



In [49]:
count_works = df_work_covariate.groupby(['individual_wikidata_id', 'category']).size().reset_index()

# Handle indivuals who have two of the same category
diverse_categories = count_works[count_works['individual_wikidata_id'].duplicated(keep=False)]
filtered_df = diverse_categories.groupby('individual_wikidata_id').filter(lambda x: x[0].nunique() == 1)
filtered_df = filtered_df.sample(frac=1, random_state=42).reset_index(drop=True)
filtered_df = filtered_df.drop_duplicates('individual_wikidata_id', keep = 'first')
filtered_df = filtered_df[['individual_wikidata_id', 'category']]

# now do the rest


# remove already handled individuals
other_categories = count_works[~count_works['individual_wikidata_id'].isin(filtered_df['individual_wikidata_id'])]
other_categories = other_categories.sort_values(['individual_wikidata_id', 0], ascending=(False, False))
other_categories = other_categories.groupby('individual_wikidata_id').head(1)
other_categories = other_categories[['individual_wikidata_id', 'category']]

final_categories = pd.concat([other_categories, filtered_df])
final_categories = final_categories.reset_index(drop=True)

df_final_category = pd.merge(df_final, final_categories, on = ['individual_wikidata_id'])
df_final_category.to_csv(DATA_PATH + "/df_indi_works_category.csv")

In [50]:
len(set(final_categories.individual_wikidata_id)) == 40732

True

,individual_wikidata_id,decade,individual_name,score,region_name,count_works,category
0,Q29436295,1660,Rombout van den Hoeye,21,Low countries,3.0,print
1,Q29436295,1660,Rombout van den Hoeye,21,Netherlands,3.0,print
2,Q29436295,1660,Rombout van den Hoeye,21,Northwestern Europe,3.0,print
3,Q29436295,1660,Rombout van den Hoeye,21,Western Europe,3.0,print
4,Q18546722,1680,Peter Cross,21,Northwestern Europe,7.0,painting
...,...,...,...,...,...,...,...
120064,Q432585,1700,Jean-Baptiste Dubos,58,France,4.0,literary work
120065,Q16063175,1840,Jane Martha St. John,9,Northwestern Europe,12.0,photograph
120066,Q16063175,1840,Jane Martha St. John,9,Western Europe,12.0,photograph
120067,Q16063175,1840,Jane Martha St. John,9,British Islands,12.0,photograph


In [51]:
df_clean_gdp = pd.read_sql_query("SELECT * FROM gdp_clean", conn)

region_code = pd.read_csv('../environnement_data/region_code_region_name.csv')
df_clean_gdp = pd.merge(df_clean_gdp, region_code, on  = 'region_code')


regions_clean = list(set(df_clean_gdp["region_name"]))

df = df[df["region_name"].isin(regions_clean)]
df = df[df["decade"] >= min(df_clean_gdp.year)]


df_final = pd.merge(df, df_count_work, on="individual_wikidata_id", how="left")
df_final = df_final.fillna(0)  # When there is no works we add 0
df_final.to_csv(DATA_PATH + "/df_indi_works_clean_gdp.csv")